# Import toolboxes used by the script and its functions

In [1]:
import os
import numpy as np
import pandas as pd
import re
from SPARQLWrapper import SPARQLWrapper, JSON
from unidecode import unidecode
from urllib2 import Request, urlopen
import json
from pandas.io.json import json_normalize
import datetime

start = datetime.datetime.now()

# Add filepaths for input file here

In [2]:
inputfile = os.path.normpath('C:/Users/rthomas/Documents/GitHub/ICES2NVS_semantic_map/example/ICES2P01_test_dset.xlsx')

# Add filepaths for mapping file here

In [3]:
mapfile = os.path.normpath('C:/Users/rthomas/Documents/GitHub/ICES2NVS_semantic_map/mappings/unmapped_substances.xlsx')
biotamap = os.path.normpath('C:/Users/rthomas/Documents/GitHub/ICES2NVS_semantic_map/mappings/biota_synonym_mapping.xlsx')

# Generate output file name

In [4]:
fileout = os.path.normpath(inputfile[:-5]+'_mapped.xlsx')

# Define functions used by the script

In [5]:
def xstr(s):
    """Function to remove non-ASCII characters from NVS results"""
    if s is None:
        return ''
    return str(unidecode(s))

def sparql_nvs_json(s):
    """Function to input a SPARQL query (s) into the NVS SPARQL endpoint"""
    sparql = SPARQLWrapper("http://vocab.nerc.ac.uk/sparql/sparql")
    sparql.setQuery(s)
    sparql.setReturnFormat(JSON)
    r = sparql.query().convert()
    return r

def S27_map():
    """Function to get NVS:S27 chemical substrances which have CAS numbers as a published mapping"""
    s =  """PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
                
            select ?codval ?prefLabel ?casurl
            where {
            <http://vocab.nerc.ac.uk/collection/S27/current/> skos:member ?url .
            ?url skos:notation ?codval .
            ?url skos:prefLabel ?prefLabel .
            ?url owl:deprecated 'false' .
            ?url owl:sameAs ?casurl .
            FILTER(regex(str(?casurl), "http://chem.sis.nlm.nih.gov/chemidplus/rn/", "i"))
            }"""                
    r = sparql_nvs_json(s)    
    list = []
    for i in range(0,len(r['results']['bindings'])):
        a = xstr(r['results']['bindings'][i]['codval']['value'].replace('SDN:S27::',''))
        b = xstr(r['results']['bindings'][i]['prefLabel']['value'])
        c = xstr(r['results']['bindings'][i]['casurl']['value'].replace('http://chem.sis.nlm.nih.gov/chemidplus/rn/',''))
        list.append([a,b,c])
    return list

def taxon_map(spcs,aphia):
    """Function to get NVS:S25 TAXON from an AphiaID and species"""
    s =  """PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
                
            select ?prefLabel
            where {
                    <http://vocab.nerc.ac.uk/collection/S25/current/> skos:member ?url .
                    ?url skos:notation ?codval .
                    ?url skos:prefLabel ?prefLabel .
                    ?url owl:deprecated 'false' .
                    FILTER(CONTAINS(?prefLabel,'%s')).
                    FILTER(CONTAINS(?prefLabel,'WoRMS %s)')).
                }""" % (spcs, aphia)
    r = sparql_nvs_json(s)
    list = []
    if len(r['results']['bindings']) == 0:
        a = 'Not available'
    else:
        label_list = []
        for i in range(0,len(r['results']['bindings'])):
            h = re.sub('\s\[.*?\]' ,'',xstr(r['results']['bindings'][i]['prefLabel']['value']))
            if h not in label_list:
                label_list.append(h)
                a = h
    list.append([a, aphia])
    return list

def S25_lookup(spcs,aphia,label):
    """Function to get NVS:S25 codval from the generated preflabel, if it exists"""
    s =  """PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
            PREFIX owl: <http://www.w3.org/2002/07/owl#>
                
            select ?codval ?prefLabel
            where {
            <http://vocab.nerc.ac.uk/collection/S25/current/> skos:member ?url .
            ?url skos:notation ?codval .
            ?url skos:prefLabel ?prefLabel .
            ?url owl:deprecated 'false' .
            FILTER(CONTAINS(?prefLabel,'%s (')).
            FILTER(CONTAINS(?prefLabel,'WoRMS %s')).
            FILTER(STRENDS(?prefLabel,'%s')).
            }""" % (spcs, aphia, label)
    r = sparql_nvs_json(s)
    list = []
    if len(r['results']['bindings']) == 0:
        a = 'No S25 term. Needs adding to NVS'
        b = "%s (ITIS: ?????: WoRMS %s) %s" % (spcs, aphia, label)
    else:
        for i in range(0,len(r['results']['bindings'])):
            a = xstr(r['results']['bindings'][i]['codval']['value'].replace('SDN:S25::',''))
            b = xstr(r['results']['bindings'][i]['prefLabel']['value'])
    list.append([a,b])
    return list

# Load ICES semantic model components for mapping to P01 semantic model from file into a Pandas DataFrame

In [9]:
inputs = pd.read_excel(inputfile)
param_combo = inputs.copy(deep=True)

# Get ICES PARAM to NVS substance mapping held in seperate Excel file
local_map = pd.read_excel(mapfile)
local_map_terms = local_map.PARAM.unique().tolist()

# Set NaNs to '-9' and add columns for mapped NVS semantic model
param_combo = param_combo.fillna('-9')
param_combo = param_combo.assign(S06_label='',
                                 S07_label='not specified',
                                 S27_label='',
                                 S27='',
                                 S02_label='',
                                 S26_label='',
                                 S25='',
                                 S25_label='',
                                 TAXON='',
                                 STAGE='not specified',
                                 GENDER='not specified',
                                 SIZE='not specified',
                                 SUBCOMPONENT='not specified',
                                 MORPHOLOGY='not specified',
                                 COLOUR='not specified',
                                 SUBGROUP='not specified')

# Remove leading or trailing spaces from the text columns
columns = param_combo.columns.tolist()
columns.remove('AphiaID')
for column in columns:
    param_combo[column] = param_combo[column].str.strip()
    
print(param_combo)

     PARGROUP                                              PRNAM         CAS  \
0          -9                                         anthracene    120-12-7   
1          -9                                     benzo[a]pyrene     50-32-8   
2          -9            2,2',4,5,5'-pentachlorobiphenyl (CB101)  37680-73-2   
3          -9            2,3,3',4,4'-pentachlorobiphenyl (CB105)  32598-14-4   
4          -9            2,3',4,4',5-pentachlorobiphenyl (CB118)  31508-00-6   
5          -9          2,2',3,4,4',5'-hexachlorobiphenyl (CB138)  35065-28-2   
6          -9          2,2',4,4',5,5'-hexachlorobiphenyl (CB153)  35065-27-1   
7          -9       2,2',3,4,4',5,5'-heptachlorobiphenyl (CB180)  35065-29-3   
8          -9                    2,4,4'-trichlorobiphenyl (CB28)   7012-37-5   
9          -9               2,2',5,5'-tetrachlorobiphenyl (CB52)  35693-99-3   
10         -9                                            cadmium   7440-43-9   
11         -9                           

# Check mapping of chemical PARAMs and NVS S27 vocabulary entries

In [8]:
# Get S27 terms that have a mapping to ICES PARAM vocabulary
q = """PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
                    
    select ?PARAM ?S27 ?S27_label 
    where {
           <http://vocab.nerc.ac.uk/collection/S27/current/> skos:member ?url .
           ?url skos:notation ?a .
           ?url skos:prefLabel ?S27_label .
           ?url owl:deprecated 'false' .
           ?url skos:related ?c .
           FILTER(regex(str(?c), "http://vocab.ices.dk/services/rdf/collection/PARAM/", "i")) .
           BIND(substr(?a,10,8) as ?S27) .
           BIND(replace(str(?c), "http://vocab.ices.dk/services/rdf/collection/PARAM/", "", "i") AS ?PARAM) .
          }"""

mapped_chems = sparql_nvs_json(q)

# Call NVS substances with CAS numbers from the SPARQL endpoint
nvs_list = S27_map()
nvs = pd.DataFrame.from_records(nvs_list,columns=['nvs_codval','nvs_preflabel','nvs_casrn'])

print(nvs)

     nvs_codval                                      nvs_preflabel  \
0      CS002244                4,4'-dichlorodiphenyldichloroethane   
1      CS002894                             total inorganic carbon   
2      CS002650                                 tributyltin cation   
3      CS002643                                    trans-nonachlor   
4      CS002503                        gamma-hexachlorocyclohexane   
5      CS000851                                          terbutryn   
6      CS001859                           2,4,4'-trichlorobiphenyl   
7      CS002335                                             barium   
8      CS002896                                      chlorophyll-a   
9      CS002258               4,4'-dichlorodiphenyltrichloroethane   
10     CS001110              2,4'-dichlorodiphenyldichloroethylene   
11     CS001719                  2,2',3,4,4',5'-hexachlorobiphenyl   
12     CS002251              4,4'-dichlorodiphenyldichloroethylene   
13     CS002664     

# Taxon, WoRMS AphiaID, ITIS TSN combination check

In [10]:
# Create a Pandas DataFrame and populate with unique combinations of Species and AphiaID from the input file
taxa_check = pd.DataFrame()
taxa_check[['Species_provided','AphiaID_provided']] = param_combo[['Species','AphiaID']][param_combo['Species']!='-9'].drop_duplicates().reset_index(drop=True)

# Iterate through each row in the DataFrame and check WoRMS as the primary source for information
for row in range(0, len(taxa_check)):
    # Check details held by WoRMS based on the scientific name provided as Species in the input file
    spcs = taxa_check.loc[row,['Species_provided']].values[0]
    url = "http://www.marinespecies.org/rest/AphiaRecordsByNames?scientificnames%5B%5D="+spcs.replace(" ","%20")+"&like=false&marine_only=true"
    request=Request(url)
    response = urlopen(request)
    if response.code==204:
        taxa_check.loc[row,'AphiaID_from_Species'] = 'No response.'
        taxa_check.loc[row,'taxon_status_from_Species'] = 'No response.'
        taxa_check.loc[row,'valid_AphiaID_from_Species'] = 'No response.'
        taxa_check.loc[row,'valid_name_from_Species'] = 'No response.'
    elif response.code==200:
        e = response.read()
        data = json.loads(e)
        taxa_check.loc[row,'AphiaID_from_Species'] = int(data[0][0]['AphiaID'])
        taxa_check.loc[row,'taxon_status_from_Species'] = data[0][0]['status']
        taxa_check.loc[row,'valid_AphiaID_from_Species'] = int(data[0][0]['valid_AphiaID'])
        taxa_check.loc[row,'valid_name_from_Species'] = data[0][0]['valid_name']
        
    # Check details held by WoRMS based on the AphiaID provided in the input file
    AphiaID = str(int(taxa_check.loc[row,['AphiaID_provided']].values[0]))
    url = "http://www.marinespecies.org/rest/AphiaRecordByAphiaID/"+AphiaID
    request=Request(url)
    response = urlopen(request)
    if response.code==204:
        taxa_check.loc[row,'name_from_AphiaID'] = 'No response.'
        taxa_check.loc[row,'taxon_status_from_AphiaID'] = 'No response.'
        taxa_check.loc[row,'valid_AphiaID_from_AphiaID'] = 'No response.'
        taxa_check.loc[row,'valid_name_from_AphiaID'] = 'No response.'
    elif response.code==200:
        e = response.read()
        data = json.loads(e)
        taxa_check.loc[row,'name_from_AphiaID'] = data['scientificname']
        taxa_check.loc[row,'taxon_status_from_AphiaID'] = data['status']
        taxa_check.loc[row,'valid_AphiaID_from_AphiaID'] = int(data['valid_AphiaID'])
        taxa_check.loc[row,'valid_name_from_AphiaID'] = data['valid_name']

# Set column to indicate if a discrepancy exists to be resolved based on AphiaIDs not matching
a = taxa_check.AphiaID_provided == taxa_check.AphiaID_from_Species
taxa_check['proceed'] = np.where(a, 'Yes', 'No')

clean_taxa = taxa_check['Species_provided'][taxa_check['proceed']=='Yes'].tolist()
query_taxa = taxa_check['Species_provided'][taxa_check['proceed']=='No'].tolist()

# Set up Pandas Dataframe for Taxon details to be populated
taxon_list = []
taxon_code = pd.DataFrame.from_records(taxon_list,columns=['taxon_preflabel','AphiaID'])

# Set up Pandas Dataframe for S25 vocab details to be populated
S25_list = []
S25_code = pd.read_excel(biotamap)

In [12]:
print(taxa_check[taxa_check['proceed']!='Yes'])

            Species_provided AphiaID_provided AphiaID_from_Species  \
2    Venerupis philippinarum           231750         No response.   
8              Psetta maxima           127149               154473   
46             Phoca hispida           159021               137083   
70                 Mysidacea           149668                 2824   
73              Raja radiata           105865               148824   
78   Clupea harengus membras           126417               714851   
91          Sebastes marinus           151324               127253   
96            Raja brachyura           105882               367297   
101           Macoma baltica           880017         No response.   

    taxon_status_from_Species valid_AphiaID_from_Species  \
2                No response.               No response.   
8                  unaccepted                     127149   
46                 unaccepted                     159021   
70                 unaccepted                     149668   

# Iterate through file for ICES combinations where a mapping does not already exist.

In [ ]:
for row in range(0,len(param_combo)):
    if param_combo.loc[row,['P01_Code']][0]=='-9':

        # Logic for contaminants in sediment mappings
        if param_combo.loc[row,['DTYPE']][0] == 'CS':
            param_combo.loc[row,['S25']] = 'BE007736'
            param_combo.loc[row,['S25_label']] = 'not applicable'
            if param_combo.loc[row,['MUNIT']][0] == '%':
                param_combo.loc[row,['S06_label']] = "Proportion"
            elif param_combo.loc[row,['MUNIT']][0][len(param_combo.loc[row,['MUNIT']][0])-1] == 'g':
                param_combo.loc[row,['S06_label']] = 'Concentration'
            else:
                param_combo.loc[row,['S06_label']] = 'Check MUNIT'
                
            if param_combo.loc[row,['BASIS']][0] == 'D':
                param_combo.loc[row,['S02_label']] = 'per unit dry weight of'
            elif param_combo.loc[row,['BASIS']][0] == 'W':
                param_combo.loc[row,['S02_label']] = 'per unit wet weight of'
            else:
                param_combo.loc[row,['S02_label']] = 'Check BASIS'
                
            if param_combo.loc[row,['MATRX']][0] == 'SEDTOT':
               param_combo.loc[row,['S26_label']] = 'sediment'
            elif param_combo.loc[row,['MATRX']][0][3:len(param_combo.loc[row,['MATRX']])] != 'TOT':
                param_combo.loc[row,['S26_label']] = 'sediment <'+param_combo.loc[row,['MATRX']][0][3:len(param_combo.loc[row]['MATRX'])] +'um'
            else:
                param_combo.loc[row,['S26_label']] = 'Check MATRX'

        # Logic for contaminants in water mappings       
        if param_combo.loc[row,['DTYPE']][0] == 'CW':
            param_combo.loc[row,['S25']] = 'BE007736'
            param_combo.loc[row,['S25_label']] = 'not applicable'
            if param_combo.loc[row,['MUNIT']][0] =='ntu':
                if param_combo.loc[row,['PARAM']][0]=='TURB':
                    param_combo.loc[row,['S06_label']] = 'Turbidity'
                    param_combo.loc[row,['S02_label']] = 'of the'        
            elif param_combo.loc[row,['MUNIT']][0] == '%':
                param_combo.loc[row,['S06_label']] = "Proportion"
            elif param_combo.loc[row,['MUNIT']][0] == 'mBq/l':
                param_combo.loc[row,['S06_label']] = "Activity"        
                param_combo.loc[row,['S02_label']] = 'per unit volume of the'
            elif param_combo.loc[row,['MUNIT']][0][len(param_combo.loc[row,['MUNIT']][0])-1] == 'g':
                param_combo.loc[row,['S06_label']] = 'Concentration'
                param_combo.loc[row,['S02_label']] = 'per unit mass of the'
            elif param_combo.loc[row,['MUNIT']][0][len(param_combo.loc[row,['MUNIT']][0])-1] == 'l':
                param_combo.loc[row,['S06_label']] = 'Concentration'
                param_combo.loc[row,['S02_label']] = 'per unit volume of the'
            else:
                param_combo.loc[row,['S06_label']] = 'Check MUNIT'
                param_combo.loc[row,['S02_label']] = 'Check MUNIT'
                        
            if param_combo.loc[row,['MATRX']][0] == 'WT':
                if param_combo.loc[row,['METPT']][0]=='-9':
                    param_combo.loc[row,['S26_label']]='water body [dissolved plus reactive particulate <unknown phase]'
                else:
                    metpt_list = param_combo.loc[row,['METPT']][0].split('~')               
                    for metpt in metpt_list:
                        if metpt in ('NF','NONE','NA','CP'):
                            param_combo.loc[row,['S26_label']]='water body [dissolved plus reactive particulate phase]'
                            continue
                        elif metpt in('GFF','GF/F','FF-GF-0.7'):
                            param_combo.loc[row,['S26_label']]='water body [dissolved plus reactive particulate <GF/F phase]'
                            continue
                        elif metpt in('GFC','GF/C','FF-GF-1.2','FF-PP-1.2'):
                            param_combo.loc[row,['S26_label']]='water body [dissolved plus reactive particulate <GF/C phase]'
                            continue
                        elif metpt in('FM-PC-0.4','FM-PC-0.45','FM-PES-0.45','FM-CN-0.45','FM-CA-0.45','PCF40','PCF45','PCF'):
                            param_combo.loc[row,['S26_label']] = 'water body [dissolved plus reactive particulate <0.4/0.45um phase]'
                            continue
                        elif metpt in('F'):
                            param_combo.loc[row,['S26_label']]='water body [dissolved plus reactive particulate <unknown phase]'
                            continue
                        elif metpt in('FM-CA-0.2'):
                            param_combo.loc[row,['S26_label']]='water body [dissolved plus reactive particulate <0.2um phase]'
                            continue
                        else:
                            if param_combo.loc[row,['S26_label']][0] == '':
                                param_combo.loc[row,['S26_label']]='Check METPT'

        # Logic for contaminants in biota mappings
        if param_combo.loc[row,['DTYPE']][0] == 'CF':
            # Set S26 label to 'biota'
            param_combo.loc[row,['S26_label']] = 'biota'
            
            if param_combo.loc[row,['CAS']][0] in ('-9','NA'):
                if param_combo.loc[row,['PARAM']][0]!='CS137':
                    param_combo.loc[row,['S06_label']] = 'Generate mapping'
                else:
                    param_combo.loc[row,['S06_label']] = 'Concentration'
            else:
                param_combo.loc[row,['S06_label']] = 'Concentration'

            # Map BASIS column for dry weight, wet weight and lipid normalisaed concentrations. Anything else requires checking.  
            if param_combo.loc[row,['BASIS']][0] == 'D':
                param_combo.loc[row,['S02_label']] = 'per unit dry weight of'
            elif param_combo.loc[row,['BASIS']][0] == 'W':
                param_combo.loc[row,['S02_label']] = 'per unit wet weight of'
            elif param_combo.loc[row,['BASIS']][0] == 'L':
                param_combo.loc[row,['S02_label']] = 'in'
                param_combo.loc[row,['S06_label']] = 'Lipid-normalised concentration'
            else:
                param_combo.loc[row,['S02_label']] = 'Check BASIS'
            
            # Map matrix of the biota to appropriate NVS S25 SUBCOMPONENT and/or STAGE. Not some constraints based on taxa type applied in the code.
            if param_combo.loc[row,['MATRX']][0] == 'WO':
                param_combo.loc[row,['SUBCOMPONENT']] = 'not specified'
            elif param_combo.loc[row,['MATRX']][0] == 'TM':
                param_combo.loc[row,['SUBCOMPONENT']] = 'muscle tissue'
            elif param_combo.loc[row,['MATRX']][0] == 'SI':
                param_combo.loc[row,['SUBCOMPONENT']] = 'not specified'
            elif param_combo.loc[row,['MATRX']][0] == 'SH':
                param_combo.loc[row,['SUBCOMPONENT']] = 'shell'
            elif param_combo.loc[row,['MATRX']][0] == 'SB':
                if param_combo.loc[row,['Species']][0] not in ('Gobius','Crangon crangon','Mysidacea'):
                    param_combo.loc[row,['SUBCOMPONENT']] = 'flesh'
                else:
                    param_combo.loc[row,['SUBCOMPONENT']] = 'Checking species-matrx combo validity with ICES.'
            elif param_combo.loc[row,['MATRX']][0] == 'RO':
                param_combo.loc[row,['STAGE']] = 'eggs'
                param_combo.loc[row,['SUBCOMPONENT']] = 'not specified'
            elif param_combo.loc[row,['MATRX']][0] == 'MU&EP':
                param_combo.loc[row,['SUBCOMPONENT']] = 'muscle tissues and skin'
            elif param_combo.loc[row,['MATRX']][0] == 'MU':
                if param_combo.loc[row,['Species']][0] == 'Loligo vulgaris':
                    param_combo.loc[row,['SUBCOMPONENT']] = 'flesh'
                else:
                    param_combo.loc[row,['SUBCOMPONENT']] = 'muscle tissue'
            elif param_combo.loc[row,['MATRX']][0] == 'LI':
                param_combo.loc[row,['SUBCOMPONENT']] = 'liver'
            elif param_combo.loc[row,['MATRX']][0] == 'KI':
                param_combo.loc[row,['SUBCOMPONENT']] = 'kidney'
            elif param_combo.loc[row,['MATRX']][0] == 'GO':
                param_combo.loc[row,['SUBCOMPONENT']] = 'gonads'
            elif param_combo.loc[row,['MATRX']][0] == 'GI':
                param_combo.loc[row,['SUBCOMPONENT']] = 'gill'
            elif param_combo.loc[row,['MATRX']][0] == 'FE':
                param_combo.loc[row,['SUBCOMPONENT']] = 'feathers'
            elif param_combo.loc[row,['MATRX']][0] == 'FA':
                param_combo.loc[row,['SUBCOMPONENT']] = 'body fat'
            elif param_combo.loc[row,['MATRX']][0] == 'EX':
                if param_combo.loc[row,['Species']][0] == 'Mytilus edulis':
                    param_combo.loc[row,['SUBCOMPONENT']] = 'shell'
                else:
                    param_combo.loc[row,['SUBCOMPONENT']] = 'Checking species-matrx combo validity with ICES.'
            elif param_combo.loc[row,['MATRX']][0] == 'EP':
                param_combo.loc[row,['SUBCOMPONENT']] = 'skin'
            elif param_combo.loc[row,['MATRX']][0] == 'EH':
                param_combo.loc[row,['STAGE']] = 'eggs'
                param_combo.loc[row,['SUBCOMPONENT']] = 'egg yolk and albumen homogenate'
            elif param_combo.loc[row,['MATRX']][0] == 'EG':
                param_combo.loc[row,['STAGE']] = 'eggs'
            elif param_combo.loc[row,['MATRX']][0] == 'BS':
                param_combo.loc[row,['SUBCOMPONENT']] = 'blood serum'
            elif param_combo.loc[row,['MATRX']][0] == 'BR':
                param_combo.loc[row,['SUBCOMPONENT']] = 'brain'
            elif param_combo.loc[row,['MATRX']][0] == 'BL':
                param_combo.loc[row,['SUBCOMPONENT']] = 'blood'
            elif param_combo.loc[row,['MATRX']][0] == 'BC':
                param_combo.loc[row,['SUBCOMPONENT']] = 'blood cells'
            elif param_combo.loc[row,['MATRX']][0] == 'BB':
                if param_combo.loc[row,['Note']][0] != 'Fish':
                    param_combo.loc[row,['SUBCOMPONENT']] = 'blubber'
                else:
                    param_combo.loc[row,['SUBCOMPONENT']] = 'Checking species-matrx combo validity with ICES.'

            # Where there is no Species - AphiaID discrepancy check the NVS to see if taxa is already published.
            if param_combo.loc[row,['Species']][0] in clean_taxa:
                spcs = param_combo.loc[row,['Species']][0]
                aphia = int(param_combo.loc[row,['AphiaID']][0])
                
                if len(taxon_code[taxon_code['AphiaID']==aphia])==0:
                    taxon_list = taxon_map(spcs, aphia)
                    taxon_code = pd.concat([taxon_code,pd.DataFrame.from_records(taxon_list,columns=['taxon_preflabel','AphiaID'])])
                    param_combo.loc[row,['TAXON']] = taxon_list[0][0]
                else:
                    param_combo.loc[row,['TAXON']] = taxon_code[taxon_code['AphiaID']==aphia].iloc[0]['taxon_preflabel']
                
                #Build S25 preflabel components for text matching
                label=''
                txn = param_combo.loc[row,['TAXON']][0]
                scp = param_combo.loc[row,['SUBCOMPONENT']][0]
                stg = param_combo.loc[row,['STAGE']][0]
                if np.any([txn == 'Not available', scp in ('Checking species-matrx combo validity with ICES.','New term needed.'), stg in ('Checking species-matrx combo validity with ICES.','New term needed.')]):
                    label = 'No term for at least one S25 model list. Needs adding to NVS'
                elif np.all([txn != 'Not available', stg == 'not specified', scp == 'not specified']):
                    label = ')'
                elif np.all([txn != 'Not available', stg == 'not specified', scp != 'not specified']):
                    label = '[Subcomponent: %s]' % (scp)
                elif np.all([txn != 'Not available', stg != 'not specified', scp == 'not specified']):
                    label = '[Stage: %s]' % (stg)
                else:
                    label = '[Stage: %s Subcomponent: %s]' % (stg, scp)
                param_combo.loc[row,['S25_label']] = (txn + " " + label).replace(") )", ")")
                
                # Look up S25 preflabel to identify appropriate S25 codval
                if label !='No term for at least one S25 model list. Needs adding to NVS':
                    if len(S25_code[S25_code['S25_preflabel']==label])==0:
                        S25_list = S25_lookup(spcs,aphia,label)
                        S25_code = pd.concat([S25_code,pd.DataFrame.from_records(S25_list,columns=['S25_codval','S25_preflabel'])])
                        param_combo.loc[row,['S25']] = S25_list[0][0]
                    else:
                        param_combo.loc[row,['S25']] = S25_code[S25_code['S25_preflabel']==label].iloc[0]['S25_codval']
                else:
                    param_combo.loc[row,['S25']] = label
            # Where there is a Species - AphiaID discrepancy continue with the next step.    
            elif param_combo.loc[row,['Species']][0] in query_taxa:
                param_combo.loc[row,['TAXON']] = 'To be resolved.'
        
        # Logic for mapping to ICES PARAM based on CAS registry number or unmapped file        
        cas = param_combo.loc[row,['CAS']][0]
        param = param_combo.loc[row,['PRNAM']][0]
        if param in local_map_terms:
            codval = local_map['NVS_CODVAL'][local_map['PARAM']==param]
            preflabel = local_map['NVS_PREFLABEL'][local_map['PARAM']==param]
            param_combo.loc[row,['S27']] = codval.iloc[0]
            param_combo.loc[row,['S27_label']] = preflabel.iloc[0]            
        elif cas in ('-9','NA'):
            param_combo.loc[row,['S27']] = 'not applicable'
            param_combo.loc[row,['S27_label']] = 'not applicable' 
        else:
            codval = nvs['nvs_codval'][nvs['nvs_casrn']==cas]
            preflabel = nvs['nvs_preflabel'][nvs['nvs_casrn']==cas]
            if len(nvs[nvs['nvs_casrn']==cas]) == 0:
                param_combo.loc[row,['S27']] = 'No S27 term. Needs adding to NVS'   
            elif len(nvs[nvs['nvs_casrn']==cas]) == 1:
                param_combo.loc[row,['S27']] = codval.iloc[0]
                param_combo.loc[row,['S27_label']] = preflabel.iloc[0]
            elif len(nvs[nvs['nvs_casrn']==cas]) > 1:
                if len(nvs[nvs['nvs_preflabel']==param]) == 1:
                    codval = nvs['nvs_codval'][nvs['nvs_preflabel']==param]
                    preflabel = nvs['nvs_preflabel'][nvs['nvs_preflabel']==param]
                    param_combo.loc[row,['S27']] = codval.iloc[0]
                    param_combo.loc[row,['S27_label']] = preflabel.iloc[0]
                elif len(nvs[nvs['nvs_preflabel']==param]) == 0:
                    if param == 'mercury':
                        param = 'total mercury'
                        codval = nvs['nvs_codval'][nvs['nvs_preflabel']==param]
                        preflabel = nvs['nvs_preflabel'][nvs['nvs_preflabel']==param]
                        param_combo.loc[row,['S27']] = codval.iloc[0]
                        param_combo.loc[row,['S27_label']] = preflabel.iloc[0]
    print("Row %s of %s complete." % (row+1,len(param_combo)))

# Replaces any problematic text characters from the NVS imported into the DataFrame
# that will cause issues when writing the output to file.
param_combo.replace(u'\xa0',u' ', regex=True, inplace=True)
param_combo.replace(u'\u2019',u"'", regex=True, inplace=True)

# Reorder columns for output
param_combo = param_combo[['PARGROUP','PRNAM','CAS','DTYPE','PARAM','MUNIT','MATRX','BASIS','METPT','METOA','Note','AphiaID','Species',
                          'P01_Code','S06_label','S07_label','S27_label','S27','S02_label','S26_label','S25','S25_label',
                          'TAXON','STAGE','GENDER','SIZE','SUBCOMPONENT','MORPHOLOGY','COLOUR','SUBGROUP']]

print("Total combinations in file = %s" % (len(param_combo)))
         

# Save the results of the ICES to NVS semantic model mapping to file.

In [ ]:
# Split out those combinations that have already been mapped to P01 in the parameter set
outputP01 = param_combo[param_combo['P01_Code']!='-9']

# Retain those combinations that have not yet been mapped to P01
output = param_combo[param_combo['P01_Code']=='-9']

# Split out SPM combinations to file where more worked needed at ICES to allow accurate mapping.
outputSPM = output[output['MATRX']=='SPM']

# Split out combinations that exclude SPM measurements 
output = output[output['MATRX']!='SPM']

# Generate list of new substances to be added to S27
S27_cols = ['PARGROUP','PARAM','PRNAM','CAS','S27_label','S27']
output_S27 = output[S27_cols][output['S27']=='No S27 term. Needs adding to NVS'].drop_duplicates().reset_index(drop=True)

# Generate list of new biological entities to be added to S25
S25_cols = ['Note','AphiaID','Species','MATRX','S25','S25_label','TAXON','STAGE','GENDER','SIZE','SUBCOMPONENT','MORPHOLOGY','COLOUR','SUBGROUP']
output_S25a = output[S25_cols][output['S25'] == 'No S25 term. Needs adding to NVS'].drop_duplicates().reset_index(drop=True)
output_S25b = output[S25_cols][output['S25'] == 'No term for at least one S25 model list. Needs adding to NVS'].drop_duplicates().reset_index(drop=True)
    
# Generate list of paramters with full set of semantic model terms for P01 matching script
output_complete = output[output['S25']!='No S25 term. Needs adding to NVS']
output_complete = output_complete[output_complete['TAXON']!='To be resolved.']
output_complete = output_complete[output_complete['S25']!='No term for at least one S25 model list. Needs adding to NVS']
output_complete = output_complete[output_complete['S27']!='No S27 term. Needs adding to NVS']
output_complete = output_complete[output_complete['S27']!='not applicable']

output_nonchem = output[output['S27']=='not applicable'].reset_index(drop=True)

taxa_discrepancy = taxa_check[taxa_check['proceed']=='No'].reset_index(drop=True)

# Create summary information
summary = pd.DataFrame([["Processing started:" , (start.strftime('%Y-%m-%d %H:%M:%S'))],
["Processing finished:" , datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')],
["" , ""],
["Rows input:", len(inputs)],
["" , ""],
["Rows with P01 provided:" , len(outputP01)],
["Rows successfully mapped:" , len(output_complete)],
["Rows unable to be mapped:" , ""],
["Because:" , ""],
["    Rows for SPM:" , len(outputSPM)],
["    Rows for non-chemical codes:" , len(output_nonchem)],
["    New chemical substances for mapping:" , len(output_S27)],
["    New biological entities for creation:" , len(output_S25a)],
["    New biological entitity semantic components:" , len(output_S25b)],
["    Taxa discrepancies:" , len(taxa_discrepancy)]])


# Save outputs as worksheets within Excel file
with pd.ExcelWriter(fileout) as writer:
    summary.to_excel(writer, sheet_name='summary', header=False, index=False)
    inputs.to_excel(writer, sheet_name='input', index=False)
    outputP01.to_excel(writer, sheet_name='known_matched', index=False)
    output_complete.to_excel(writer, sheet_name='mapped', index=False)
    outputSPM.to_excel(writer, sheet_name='SPM_codes', index=False)
    output_nonchem.to_excel(writer, sheet_name='nonchemical_codes', index=False)
    output_S27.to_excel(writer, sheet_name='new_S27', index=False)
    output_S25a.to_excel(writer, sheet_name='new_S25', index=False)
    output_S25b.to_excel(writer, sheet_name='new_S25_component', index=False)
    taxa_discrepancy.to_excel(writer, sheet_name='taxa_discrepancies', index=False)
    